In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

In [2]:
## Read data
import os
print(os.getcwd())

/Users/kunalchakraborty/Downloads/industry_project_HQ


## Fama-French Factors

### Start with 3 factors

In [8]:
### Preprocess 3 factors dataset
ff3 = pd.read_csv("data/ff3.csv",skiprows=3,nrows=693, index_col='Unnamed: 0',)
ff3.index = pd.to_datetime(ff3.index,format='%Y%m')

In [11]:
## 5 factors
ff5 = pd.read_csv("data/ff5.CSV",skiprows=3,nrows=693, index_col='Unnamed: 0',)
ff5.index = pd.to_datetime(ff5.index,format='%Y%m')

## Amit Goyal's factors

In [17]:
## Load index data and construct the features
data = pd.read_csv("data/index_data.csv")

In [18]:
## As a starter, use Amit goyals data to create features and experiment
## Compute net returns on S&P
data["temp"] = data["Index"] + data["D12"]/12
data["returns"] = data["temp"]/data["Index"].shift()
data["returns"] = data["returns"] - 1
data.drop("temp",axis=1,inplace=True)

In [20]:
# equity premium, i.e., the total rate of return on the stock market minus the prevailing short-term interest rate.
data['premium'] = data['returns'] - data['Rfree']

# Dividend Price Ratio (d/p) is the difference between the log of dividends and the log of prices. 
data['d/p'] = np.log(data['D12']) - np.log(data['Index'])

# Dividend Yield (d/y) is the difference between the log of dividends and the log of lagged prices.
data['d/y'] = np.log(data['D12'])- np.log(data['Index'].shift(1))

# Earnings Price Ratio (e/p) is the difference between the log of earnings and the log of prices.
data['e/p'] = np.log(data['E12']) - np.log(data['Index'])

# Dividend Payout Ratio (d/e) is the difference between the log of dividends and the log of earnings.
data['d/e'] = np.log(data['D12']) - np.log(data['E12'])

# Term Spread (tms) is the difference between the long term yield on government bonds and the T-bill.
data['tms'] = data['lty'] - data['tbl']

# Default Yield Spread (dfy): is the difference between BAA- and AAA- rated cor- porate bond yields.
data['dfy'] = data['BAA'] - data['AAA']

# Default Return Spread (dfr): is the difference between the return on long-term corporate bonds and returns on the long-term government bonds.
data['dfr'] = data['corpr'] - data['ltr']

In [21]:
data.head()

,date,Index,D12,E12,b/m,tbl,AAA,BAA,lty,ntis,...,CRSP_SPvwx,returns,premium,d/p,d/y,e/p,d/e,tms,dfy,dfr
0,1959-01-01,55.42,1.75667,2.96333,0.502896,0.0282,0.0412,0.0487,0.0408,0.027736,...,0.005395,NaN,NaN,-3.451521,NaN,-2.928627,-0.522894,0.0126,0.0075,0.0052
1,1959-02-01,55.41,1.76333,3.03667,0.494946,0.0270,0.0414,0.0489,0.0402,0.027899,...,0.002210,0.002471,0.000571,-3.447556,-3.447736,-2.903999,-0.543557,0.0132,0.0075,0.0009
2,1959-03-01,55.44,1.77000,3.11000,0.516860,0.0280,0.0413,0.0485,0.0403,0.024984,...,0.000826,0.003203,0.001003,-3.444322,-3.443781,-2.880679,-0.563643,0.0123,0.0072,-0.0100
3,1959-04-01,57.59,1.77667,3.20667,0.498597,0.0295,0.0423,0.0486,0.0414,0.024361,...,0.039093,0.041451,0.039451,-3.478608,-3.440561,-2.888116,-0.590492,0.0119,0.0063,-0.0055
4,1959-05-01,58.68,1.78333,3.30333,0.483077,0.0284,0.0437,0.0496,0.0417,0.025887,...,0.019043,0.021507,0.019307,-3.493617,-3.474867,-2.877168,-0.616449,0.0133,0.0059,-0.0109


In [22]:
data.to_csv("data/ag_transformed_features.csv",index=False)